<a href="https://colab.research.google.com/github/sufpro/colabnotebooks/blob/main/fromDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages
!pip install transformers torch

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm

# 1. Expanded dataset with 20 examples per intent
data_pairs = [
    # Hotel (20 examples)
    ("book hotel in Tokyo", "hotel"),
    ("reserve hotel Paris", "hotel"),
    ("find hotel London", "hotel"),
    ("need hotel Berlin", "hotel"),
    ("stay in Dubai hotel", "hotel"),
    ("hotel reservation New York", "hotel"),
    ("book room Rome", "hotel"),
    ("find accommodations Sydney", "hotel"),
    ("need lodging Singapore", "hotel"),
    ("book stay Bali", "hotel"),
    ("reserve luxury hotel Tokyo", "hotel"),
    ("find cheap hotel Paris", "hotel"),
    ("need 5-star hotel London", "hotel"),
    ("book beachfront hotel", "hotel"),
    ("find pet-friendly hotel", "hotel"),
    ("resort booking needed", "hotel"),
    ("need hotel with pool", "hotel"),
    ("book business hotel", "hotel"),
    ("find family hotel", "hotel"),
    ("need hotel near airport", "hotel"),

    # Flight (20 examples)
    ("book flight London", "flight"),
    ("need flight Paris", "flight"),
    ("find flight Tokyo", "flight"),
    ("search flight New York", "flight"),
    ("book tickets Berlin", "flight"),
    ("need plane Dubai", "flight"),
    ("find airfare Rome", "flight"),
    ("search air ticket Sydney", "flight"),
    ("book plane Singapore", "flight"),
    ("find airline Bali", "flight"),
    ("search cheap flights", "flight"),
    ("need business class ticket", "flight"),
    ("find last minute flight", "flight"),
    ("book roundtrip flight", "flight"),
    ("need economy class ticket", "flight"),
    ("search direct flights", "flight"),
    ("find flights with wifi", "flight"),
    ("book window seat flight", "flight"),
    ("need flight with meals", "flight"),
    ("find nonstop flight", "flight"),

    # Query (20 examples)
    ("what hotels available", "query"),
    ("show vacation options", "query"),
    ("find travel deals", "query"),
    ("what flights exist", "query"),
    ("show travel packages", "query"),
    ("what are options", "query"),
    ("find trip suggestions", "query"),
    ("search holiday deals", "query"),
    ("what Tokyo options", "query"),
    ("show available trips", "query"),
    ("what are best hotels", "query"),
    ("show cheapest flights", "query"),
    ("find vacation ideas", "query"),
    ("what packages available", "query"),
    ("show family trip options", "query"),
    ("find romantic getaways", "query"),
    ("what business travel options", "query"),
    ("show last minute deals", "query"),
    ("find all inclusive deals", "query"),
    ("what are top destinations", "query"),

    # Business (20 examples)
    ("business trip Singapore", "business"),
    ("corporate travel Dubai", "business"),
    ("conference travel plan", "business"),
    ("business travel London", "business"),
    ("executive trip Paris", "business"),
    ("company travel Tokyo", "business"),
    ("sales meeting travel", "business"),
    ("business flight New York", "business"),
    ("corporate lodging Berlin", "business"),
    ("executive transport Rome", "business"),
    ("arrange business hotel", "business"),
    ("need corporate flight", "business"),
    ("book meeting venue", "business"),
    ("plan conference travel", "business"),
    ("organize executive trip", "business"),
    ("arrange business car", "business"),
    ("need team travel", "business"),
    ("book business lounge", "business"),
    ("find corporate rates", "business"),
    ("arrange client meeting", "business"),

    # Car (20 examples)
    ("rent car Rome", "car"),
    ("need SUV Miami", "car"),
    ("book convertible LA", "car"),
    ("find minivan rental", "car"),
    ("rent vehicle Dubai", "car"),
    ("search luxury car", "car"),
    ("book compact Tokyo", "car"),
    ("find van rental", "car"),
    ("rent auto Paris", "car"),
    ("need business car", "car"),
    ("find cheap rental", "car"),
    ("book premium vehicle", "car"),
    ("need family car", "car"),
    ("search hybrid rental", "car"),
    ("find electric car", "car"),
    ("rent truck for move", "car"),
    ("book sports car", "car"),
    ("need long term rental", "car"),
    ("find one-way rental", "car"),
    ("search airport rental", "car")
]

# 2. Initialize larger model with dropout
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Add dropout to prevent overfitting
model.config.dropout_rate = 0.1  # Added dropout
model.config.attention_dropout = 0.1  # Attention dropout

# 3. Enhanced Dataset class
class IntentDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, intent = self.data[idx]
        inputs = self.tokenizer(
            f"classify intent: {text}",
            max_length=32,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        targets = self.tokenizer(
            intent,
            max_length=8,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': targets['input_ids'].flatten()
        }

# 4. Training setup with early stopping
dataset = IntentDataset(data_pairs, tokenizer)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training with early stopping
best_loss = float('inf')
patience = 3
no_improvement = 0

for epoch in range(100):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
            'labels': batch['labels'].to(device)
        }
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Loss: {avg_loss:.4f}")

    # Early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        no_improvement = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f"No improvement for {patience} epochs, stopping early")
            break

# Load best model
model.load_state_dict(torch.load('best_model.pt'))

# 5. Enhanced prediction with partial matching
def predict_intent(text):
    model.eval()
    inputs = tokenizer(
        f"classify intent: {text}",
        return_tensors='pt',
        max_length=32,
        padding='max_length',
        truncation=True
    ).to(device)

    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=8,
        num_beams=3,
        temperature=0.1,
        early_stopping=True
    )

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Enhanced validator with partial matching
    intent_map = {
        'hotel': ['hotel', 'stay', 'room', 'reservation', 'lodging'],
        'flight': ['flight', 'plane', 'airline', 'ticket', 'airfare'],
        'query': ['query', 'option', 'what', 'show', 'find', 'search'],
        'business': ['business', 'corporate', 'executive', 'meeting', 'conference'],
        'car': ['car', 'rent', 'vehicle', 'auto', 'SUV']
    }

    for intent, keywords in intent_map.items():
        if any(keyword in prediction.lower() for keyword in keywords):
            return intent

    return "unknown"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

Epoch 8: 100%|██████████| 13/13 [01:27<00:00,  6.72s/it]


Epoch 8 Loss: 1.8950


Epoch 9: 100%|██████████| 13/13 [01:32<00:00,  7.10s/it]


Epoch 9 Loss: 1.4839


Epoch 10:  69%|██████▉   | 9/13 [01:02<00:27,  6.87s/it]

In [ ]:
# 6. Comprehensive testing
test_cases = [
    ("I need to book a hotel in Tokyo", "hotel"),
    ("Find me flights to London", "flight"),
    ("What are my hotel options?", "query"),
    ("Arrange business travel to Paris", "business"),
    ("Rent a luxury car in Dubai", "car"),
    ("Book me a flight tomorrow", "flight"),
    ("Need accommodations in New York", "hotel"),
    ("Show me business class flights", "query"),
    ("Plan corporate trip to Berlin", "business"),
    ("Find me an SUV rental", "car"),
    ("What are the best hotels?", "query"),
    ("Need plane tickets ASAP", "flight"),
    ("Book conference hotel", "business"),
    ("Search for convertible rentals", "car"),
    ("Show vacation packages", "query")
]

print("\nFinal Test Results:")
print("{:<40} {:<10} {:<10}".format("INPUT", "EXPECTED", "PREDICTED"))
print("-" * 60)
for text, expected in test_cases:
    predicted = predict_intent(text)
    print("{:<40} {:<10} {:<10}".format(
        text[:35] + "..." if len(text) > 35 else text,
        expected,
        predicted
    ))